1. Import packages, register Glow, configure credentials to access 1000 genomes dataset from Azure Genomics Data Lake and read data for chromosome 22.

In [1]:
from pyspark.sql.functions import explode, col, lit, xxhash64

import glow
spark = glow.register(spark)

spark.conf.set(
  'fs.azure.sas.dataset.dataset1000genomes.blob.core.windows.net',
  'sv=2019-10-10&si=prod&sr=c&sig=9nzcxaQn0NprMPlSh4RhFQHcXedLQIcFgbERiooHEqM%3D')

source = 'wasbs://dataset@dataset1000genomes.blob.core.windows.net/release/20130502/ALL.chr22*.vcf.gz'

df = spark.read.\
  format('vcf').\
  option('includeSampleIds', True).\
  option('flattenInfoFields', True).\
  load(source)

We will use a subset of chromosome 22 to keep time and cost low. If you want to do format conversion for the whole chromosome 22 dataset you still can use this notebook but might need bigger cluster.

In [2]:
data=df.limit(10000)

Take a look at regular sites

In [3]:
data.where('INFO_MULTI_ALLELIC = FALSE').show(2)

and multiallelic sites.

In [6]:
data.where('INFO_MULTI_ALLELIC = TRUE').show(2)

2. Transform data: add _hashId_ column (hash is built on all columns except genotypes and might be used as unique id for variants), explode on genotypes and flatten genotypes column.

In [4]:
hashCols = list(set(data.columns) - {'genotypes'})
dataHashed = data.withColumn('hashId', xxhash64(*hashCols))

dataExploded = dataHashed.withColumn('genotypes', explode('genotypes'))

def flattenStructFields(df):
  flat_cols = [c[0] for c in df.dtypes if c[1][:6] != 'struct']
  nested_cols = [c[0] for c in df.dtypes if c[1][:6] =='struct']
  flat_df = df.select(flat_cols + 
                     [col(nc+'.'+c).alias(nc+'_'+c)
                     for nc in nested_cols
                     for c in df.select(nc+'.*').columns])
  return flat_df

dataExplodedFlatten = flattenStructFields(dataExploded)

Take a look at transformed dataset: regular sites


In [5]:
dataExplodedFlatten.where('INFO_MULTI_ALLELIC = FALSE').show(2)

and multiallelic sites.

In [32]:
dataExplodedFlatten.where('INFO_MULTI_ALLELIC = TRUE').show(2)

3. To save the data you need to provide names for output storage account, container and relative path, and set _outputPath_

In [6]:
outputStorageAccount =  'Your account name' # replace with your account name
outputContainer = 'Your container name' # replace with your container name
outputDir = 'Your path' # replace with your relative path

**Option 1:** If you use Azure Synapse Analytics and _outputStorageAccount_ is a primary (default) storage account in your Synapse workspace, run cell below

Primary storage account is ADLS Gen2, so we use 
```
adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path)
```

In [34]:
outputPath = 'abfss://%s@%s.dfs.core.windows.net/%s' % (outputContainer, outputStorageAccount, outputDir) 

**Option 2:** If you want to save data to Azure Blob Storage, provide SAS token for output container and run cell below. It works for Azure Databricks and Azure Synapse Analytics. 

For Azure Blob Storage we use  
```
wasb_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
```

In [7]:
outputSAS = 'Your SAS token' # replace with your SAS token
spark.conf.set(
  'fs.azure.sas.' + outputContainer + '.' + outputStorageAccount + '.blob.core.windows.net', outputSAS)

outputPath = 'wasbs://%s@%s.blob.core.windows.net/%s' % (outputContainer, outputStorageAccount, outputDir)

 To learn more about Azure Blob Storage and Azure Data Lake Storage (ADLS) Gen2 and different ways to access them check documentation: 

 - Azure Databricks - https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/adls-gen2/ and https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/azure-storage
 - Azure Synapse Analytics - https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/microsoft-spark-utilities?pivots=programming-language-python

4. Write data in .parquet format to your storage account: original data

In [8]:
sink = outputPath + '/original/chr22'
dataHashed.write. \
  mode('overwrite'). \
  format('parquet'). \
  save(sink)

and transformed data

In [9]:
sink = outputPath + '/flattened/chr22' 

dataExplodedFlatten.write. \
  mode('overwrite'). \
  format('parquet'). \
  save(sink)